# Real-Time Transcriptions

**Real-time Speech Recognition (Speech To Text-only)**

The Real-Time Transcriptions API and TwiML <Transcription> allow customers to enable real-time transcription of Twilio Voice calls, including Voice SDK, PSTN, WhatsApp Calling or any other Voice API-managed call.

Real-Time Transcriptions can be enabled for all Twilio Account recordings (as a part of a Voice Intelligence service) or for specific calls, using the TwiML <Transcription>.

**Note:** For Real-Time Transcription of non-Twilio Voice calls or external recordings, please check Voice Intelligence.

### Setup up your environment

In [ ]:
%pip install twilio dotenv

In [9]:
# Download the helper library from https://www.twilio.com/docs/python/install
import os
from dotenv import load_dotenv
from twilio.rest import Client

# Set the environment variables. See http://twil.io/secure
load_dotenv()

# Find your Account SID and Auth Token at twilio.com/console
# We recommend using API keys instead of Auth Tokens for production accounts
account_sid = os.environ["TWILIO_ACCOUNT_SID"]
auth_token = os.environ["TWILIO_AUTH_TOKEN"]

# Create a Twilio client using API keys
api_key = os.environ["TWILIO_API_KEY"]
api_secret = os.environ["TWILIO_API_SECRET"]

client = Client(account_sid, auth_token)


### Start Real-Time Transcription on Active Call

**Test Call Setup (transcription details in following sections)**


In [ ]:

# Call details
to_number = "+5511xxxxxxxx"
from_number = "+5511xxxxxxxx"  # your Twilio number

# Start a Twilio Call
call = client.calls.create(
    to=to_number,
    from_=from_number,
    url="http://demo.twilio.com/docs/voice.xml", # URL to TwiML instructions, can be replaced with your own URL or TwiML Bin URL (or replaced by a TwiML object and the `twiml` attribute).
)

print(call.sid)

**Real-time transcription of Active Calls**

In [ ]:
# Start a real-time transcription for an **active call** using the Transcriptions REST API: https://www.twilio.com/docs/voice/api/realtime-transcription-resource#start-a-real-time-transcription

transcription = client.calls(
    "CA6e5c4de4863a9549ed4fff2cf3d2e103"
).transcriptions.create(
    status_callback_url="https://webhook.site/084c2cbb-4a4a-47e8-8eb4-d6aaa537c49b",
    status_callback_method="POST",
)

print(transcription.sid)

### Start a New Call with Real-Time Transcription from the start

Please note Twiio Voice outbound calls **_require_** the `to`, `from` and `url` or `twiml` parameters to be set.

The URL must return TwiML instructions on how to handle the call.

**IMPORTANT:** Just starting the transcription service without a `<Dial>`, `<Connect>`, `<Enqueue>` or similar TwiML instruction will cause the call to fail, as it needs to be connected to another party.

The other party can be it a Studio Flow, an AI Assistant or any other Connect based integration, a PSTN Phone Number, a SIP endpoint or a Twilio Client.

In [ ]:
# This code demonstrates how to create an outbound call with Twilio and start a real-time transcription using the Twilio Python library.


# Call details
to_number = "+5511xxxxxxxx"
from_number = "+5511xxxxxxxx"
workflow_sid = "WWxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # Your Twilio TaskRouter Workflow SID

# Transcription parameters
status_callback_url = "https://your-webhook-url.com/callback"  # URL to receive transcription status updates
name = "Transcription"
language_code = "es-US"
track = "both_tracks"
inbound_label = "cliente"
outbound_label = "agente"
transcription_engine = "google"
intelligence_service = "GAxxxxxxxxxxxx" # Voice Intelligence Service SID, optional. Use only for storing Transcriptions in Twilio or using Twilio Language Operators.

# Build TwiML as a string
twiml = f"""
<Response>
  <Start>
    <Transcription 
      statusCallbackUrl="{status_callback_url}"
      name="{name}"
      languageCode="{language_code}"
      track="{track}"
      inboundTrackLabel="{inbound_label}"
      outboundTrackLabel="{outbound_label}"
      transcriptionEngine="{transcription_engine}"
      intelligenceService="{intelligence_service}" />
  </Start>
  <Enqueue workflowSid="{workflow_sid}" />
</Response>
""".strip()

# Create the outbound call with inline TwiML
call = client.calls.create(
    twiml=twiml,
    to=to_number,
    from_=from_number
)

print(f"Call SID: {call.sid}")



**Sample TwiML instructions for testing**

#### Start transcription and Dial a Twilio Voice Client


In [ ]:

identity = "user_jane"
FirstName = "Jane"
LastName = "Doe"

client_twiml = f"""
<Response>
  <Start>
    <Transcription 
      statusCallbackUrl="{status_callback_url}"
      name="{name}"
      languageCode="{language_code}"
      track="{track}"
      inboundTrackLabel="{inbound_label}"
      outboundTrackLabel="{outbound_label}"
      transcriptionEngine="{transcription_engine}"
      intelligenceService="{intelligence_service}" />
  </Start>
   <Dial>
     <Client>
        <Identity>{identity}</Identity>
        <Parameter name="FirstName" value ="{FirstName}"/>
        <Parameter name="LastName" value ="{LastName}" />
      </Client>
    </Dial>
</Response>
""".strip()


#### Start transcription and Dial a PSTN phone number


In [ ]:


# Start transcription and Dial a PSTN Phone Number

pstn_phone_numer = "+5511xxxxxxxx"

pstn_twiml = f"""
<Response>
  <Start>
    <Transcription 
      statusCallbackUrl="{status_callback_url}"
      name="{name}"
      languageCode="{language_code}"
      track="{track}"
      inboundTrackLabel="{inbound_label}"
      outboundTrackLabel="{outbound_label}"
      transcriptionEngine="{transcription_engine}"
      intelligenceService="{intelligence_service}" />
  </Start>
   <Dial>{pstn_phone_numer}</Dial>
</Response>
""".strip()



#### Start transcription and Dial a SIP endpoint


In [ ]:

# Start transcription and Dial a SIP endpoint
sip_endpoint = "sip:your-sip-uri"
region = "br1"  # Optional, specify the region for the SIP endpoint https://www.twilio.com/docs/voice/twiml/sip#sip-uri-region


pstn_twiml = f"""
<Response>
  <Start>
    <Transcription 
      statusCallbackUrl="{status_callback_url}"
      name="{name}"
      languageCode="{language_code}"
      track="{track}"
      inboundTrackLabel="{inbound_label}"
      outboundTrackLabel="{outbound_label}"
      transcriptionEngine="{transcription_engine}"
      intelligenceService="{intelligence_service}" />
  </Start>
   <Dial>{sip_endpoint}</Dial>
</Response>
""".strip()